<a href="https://colab.research.google.com/github/Cris-0506/exercicios_ML/blob/main/Analise_credito_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Análise dos dados de crédito utilizando Naive Bayes. Exercício do curso de Machine Learning e Data Science da Udemy.

Base de dados retirada do Kaggle (www.kaggle.com/laotse/credit-risk-dataset)

In [40]:
import pandas as pd

In [41]:
import numpy as np

In [42]:
dados = pd.read_csv('/content/credit_data.csv', sep=',', encoding = 'iso-8859-1')

In [43]:
dados

,clientid,income,age,loan,default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1
...,...,...,...,...,...
1995,1996,59221.044874,48.518179,1926.729397,0
1996,1997,69516.127573,23.162104,3503.176156,0
1997,1998,44311.449262,28.017167,5522.786693,1
1998,1999,43756.056605,63.971796,1622.722598,0


In [44]:
dados.shape

(2000, 5)

In [45]:
dados.dtypes

clientid      int64
income      float64
age         float64
loan        float64
default       int64
dtype: object

In [46]:
dados.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.807559,4444.369695,0.141500
std,577.494589,14326.327119,13.624469,3045.410024,0.348624
min,1.000000,20014.489470,-52.423280,1.377630,0.000000
25%,500.750000,32796.459717,28.990415,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


In [47]:
dados.isnull().sum()

clientid    0
income      0
age         3
loan        0
default     0
dtype: int64

In [48]:
dados.head()

,clientid,income,age,loan,default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


In [49]:
dados_credito = dados.rename (columns = {'clientid': 'cliente', 'income':'renda', 'age': 'idade', 'loan': 'divida', 'default': 'inadimplencia'})

In [50]:
dados_credito.head()

,cliente,renda,idade,divida,inadimplencia
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


In [51]:
# Existem dados inconsistentes, como os dados relacionados à idade que são valores negativos.
dados_credito.describe()

,cliente,renda,idade,divida,inadimplencia
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.807559,4444.369695,0.141500
std,577.494589,14326.327119,13.624469,3045.410024,0.348624
min,1.000000,20014.489470,-52.423280,1.377630,0.000000
25%,500.750000,32796.459717,28.990415,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


In [52]:
dados_credito[dados_credito['idade']<0]

,cliente,renda,idade,divida,inadimplencia
15,16,50501.726689,-28.218361,3977.287432,0
21,22,32197.620701,-52.423280,4244.057136,0
26,27,63287.038908,-36.496976,9595.286289,0


In [53]:
# Preenchendo os valores de idade que são negativos com a média das idades.

In [54]:
dados_credito.mean()

cliente           1000.500000
renda            45331.600018
idade               40.807559
divida            4444.369695
inadimplencia        0.141500
dtype: float64

In [55]:
# Fazendo a média sem contar com os valores negativos.
dados_credito['idade'][dados_credito['idade']>0].mean()

40.92770044906149

In [56]:
dados_credito.loc[dados_credito['idade']<0, 'idade'] = 40.92

In [57]:
dados_credito.loc[dados_credito['idade'] <0]

,cliente,renda,idade,divida,inadimplencia


In [58]:
dados_credito.head(27)

,cliente,renda,idade,divida,inadimplencia
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1
5,6,24904.064140,57.471607,15.498598,0
6,7,48430.359613,26.809132,5722.581981,0
7,8,24500.141984,32.897548,2971.003310,1
8,9,40654.892537,55.496853,4755.825280,0
9,10,25075.872771,39.776378,1409.230371,0


In [59]:
# Verificando valores faltantes.
dados_credito.isnull().sum()

cliente          0
renda            0
idade            3
divida           0
inadimplencia    0
dtype: int64

In [60]:
dados_credito.loc[pd.isnull(dados_credito['idade'])]

,cliente,renda,idade,divida,inadimplencia
28,29,59417.805406,NaN,2082.625938,0
30,31,48528.852796,NaN,6155.784670,0
31,32,23526.302555,NaN,2862.010139,0


In [61]:
# Preenchendo os valores faltantes com a média das idades.
dados_credito['idade'].fillna(dados_credito['idade'].mean(), inplace=True)

In [62]:
dados_credito.loc[dados_credito['cliente'].isin([29,30,31])]

,cliente,renda,idade,divida,inadimplencia
28,29,59417.805406,40.927689,2082.625938,0
29,30,58842.891308,54.510948,10871.186790,0
30,31,48528.852796,40.927689,6155.784670,0


In [63]:
dados_credito

,cliente,renda,idade,divida,inadimplencia
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1
...,...,...,...,...,...
1995,1996,59221.044874,48.518179,1926.729397,0
1996,1997,69516.127573,23.162104,3503.176156,0
1997,1998,44311.449262,28.017167,5522.786693,1
1998,1999,43756.056605,63.971796,1622.722598,0


In [64]:
# Separação dos dados previsores e da classe.

In [65]:
x_credito = dados_credito.iloc[:,1:4].values

In [66]:
type(x_credito)

numpy.ndarray

In [67]:
y_credito = dados_credito.iloc[:, 4].values

In [68]:
type(y_credito)

numpy.ndarray

In [69]:
#Padronizando os dados
from sklearn.preprocessing import StandardScaler
dados_padronizados = StandardScaler()
x_credito = dados_padronizados.fit_transform(x_credito)

In [70]:
x_credito.shape

(2000, 3)

In [71]:
# Divisão dos atributos previsores em treinamento e teste.
from sklearn.model_selection import train_test_split

In [72]:
from pandas.core.common import random_state
x_credito_treino, x_credito_teste, y_credito_treino, y_credito_teste = train_test_split (x_credito, y_credito, test_size = 0.25, random_state = 0)


In [73]:
x_credito_treino.shape, y_credito_treino.shape

((1500, 3), (1500,))

In [74]:
x_credito_teste.shape, y_credito_teste.shape

((500, 3), (500,))

In [75]:
# Fazendo e testando o algoritmo para análise de crédito.
base_risco_credito = pd.read_csv('/content/risco_credito.csv')

In [76]:
base_risco_credito

,historia,divida,garantias,renda,risco
0,ruim,alta,nenhuma,0_15,alto
1,desconhecida,alta,nenhuma,15_35,alto
2,desconhecida,baixa,nenhuma,15_35,moderado
3,desconhecida,baixa,nenhuma,acima_35,alto
4,desconhecida,baixa,nenhuma,acima_35,baixo
5,desconhecida,baixa,adequada,acima_35,baixo
6,ruim,baixa,nenhuma,0_15,alto
7,ruim,baixa,adequada,acima_35,moderado
8,boa,baixa,nenhuma,acima_35,baixo
9,boa,alta,adequada,acima_35,baixo


In [77]:
x_risco_credito = base_risco_credito.iloc[:,0:4].values

In [78]:
x_risco_credito

array([['ruim', 'alta', 'nenhuma', '0_15'],
       ['desconhecida', 'alta', 'nenhuma', '15_35'],
       ['desconhecida', 'baixa', 'nenhuma', '15_35'],
       ['desconhecida', 'baixa', 'nenhuma', 'acima_35'],
       ['desconhecida', 'baixa', 'nenhuma', 'acima_35'],
       ['desconhecida', 'baixa', 'adequada', 'acima_35'],
       ['ruim', 'baixa', 'nenhuma', '0_15'],
       ['ruim', 'baixa', 'adequada', 'acima_35'],
       ['boa', 'baixa', 'nenhuma', 'acima_35'],
       ['boa', 'alta', 'adequada', 'acima_35'],
       ['boa', 'alta', 'nenhuma', '0_15'],
       ['boa', 'alta', 'nenhuma', '15_35'],
       ['boa', 'alta', 'nenhuma', 'acima_35'],
       ['ruim', 'alta', 'nenhuma', '15_35']], dtype=object)

In [79]:
y_risco_credito = base_risco_credito.iloc[:,4].values

In [80]:
y_risco_credito

array(['alto', 'alto', 'moderado', 'alto', 'baixo', 'baixo', 'alto',
       'moderado', 'baixo', 'baixo', 'alto', 'moderado', 'baixo', 'alto'],
      dtype=object)

In [81]:
# Transformando os dados categóricos em numéricos.
from sklearn.preprocessing import LabelEncoder
label_encoder_historia = LabelEncoder()
label_encoder_divida = LabelEncoder()
label_encoder_garantia = LabelEncoder()
label_encoder_renda = LabelEncoder()

In [82]:
x_risco_credito[:,0] = label_encoder_historia.fit_transform(x_risco_credito[:,0])
x_risco_credito[:,1] = label_encoder_divida.fit_transform(x_risco_credito[:,1])
x_risco_credito[:,2] = label_encoder_garantia.fit_transform(x_risco_credito[:,2])
x_risco_credito[:,3] = label_encoder_renda.fit_transform(x_risco_credito[:,3])

In [83]:
x_risco_credito

array([[2, 0, 1, 0],
       [1, 0, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 2],
       [1, 1, 1, 2],
       [1, 1, 0, 2],
       [2, 1, 1, 0],
       [2, 1, 0, 2],
       [0, 1, 1, 2],
       [0, 0, 0, 2],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [0, 0, 1, 2],
       [2, 0, 1, 1]], dtype=object)

In [84]:
# Salvando a base de dados.
import pickle
with open ('risco_credito.pkl','wb') as f:
  pickle.dump ([x_risco_credito, y_risco_credito], f)

In [85]:
# Criando o algoritmo Naive Bayes
from sklearn.naive_bayes import GaussianNB

In [86]:
naive_risco_credito = GaussianNB ()

In [87]:
# Gerando a tabela de probabilidades.
naive_risco_credito.fit(x_risco_credito, y_risco_credito)

GaussianNB()

In [88]:
# Fazendo previsões de dois novos cliente.
# Cliente a: historia boa (0); divida alta (0); garantias nenhuma (1) e renda > 35000(2)
# Cliente b: historia ruim (2); divida alta (0); garatias adequada (0) e renda < 15000(0)

previsao = naive_risco_credito.predict([[0,0,1,2],[2,0,0,0]])


In [89]:
previsao

array(['baixo', 'moderado'], dtype='<U8')

In [90]:
# Observando os dados e as probabilidades apriori.
naive_risco_credito.class_count_


array([6., 5., 3.])

In [91]:
naive_risco_credito.class_prior_

array([0.42857143, 0.35714286, 0.21428571])